# Jane Street Market Prediction (#4.1)
## Hyper-parameter tuning with HyperOpt

Notebook Navigation<br>
[All](https://www.kaggle.com/wendellavila/janestreet-index/) | [#1](https://www.kaggle.com/wendellavila/janestreet-model-selection/) | [#2.1](https://www.kaggle.com/wendellavila/janestreet-preprocessing-selection) | [#2.2](https://www.kaggle.com/wendellavila/janestreet-data-preprocessing) | [#3](https://www.kaggle.com/wendellavila/janestreet-regularization-selection) | [#4.1](https://www.kaggle.com/wendellavila/janestreet-hyperparameter-tuning) | [#4.2](https://www.kaggle.com/wendellavila/janestreet-hyperparameter-evaluation) | [#5.1](https://www.kaggle.com/wendellavila/janestreet-pca) | [#5.2](https://www.kaggle.com/wendellavila/janestreet-autoencoder) | [#5.3](https://www.kaggle.com/wendellavila/janestreet-dimensionality-reduction-evaluation) |[#6](https://www.kaggle.com/wendellavila/janestreet-ensemble)

## Imports

In [1]:
import os
import glob
import IPython

import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 150)
%matplotlib inline

import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras.models import load_model
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

import sys
import time
import hyperopt
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, space_eval
from hyperopt.pyll.base import scope
from hyperopt.pyll.stochastic import sample


## Reading Data

In [2]:
#data loaded from another notebook
#already preprocessed and downsized for faster loading
data = pd.read_pickle('../input/janestreet-data-preprocessing/train-ffil-indicator.pkl')
features = [c for c in data.columns if 'feature' in c]
resp_cols = [col for col in data.columns if 'resp' in col]

## Misc

In [3]:
# weighted average as per Donate et al.'s formula
# https://doi.org/10.1016/j.neucom.2012.02.053
def weighted_average(a):
    w = []
    n = len(a)
    for j in range(1, n + 1):
        j = 2 if j == 1 else j
        w.append(1 / (2**(n + 1 - j)))
    return np.average(a, weights = w)

In [4]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
      def on_train_end(*args, **kwargs):
        IPython.display.clear_output(wait = True)

## Utility Score

In [5]:
def utility_score(d, w, r, a):
    Pi = np.bincount(d, w * r * a)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / len(Pi))
    u = min(max(t, 0), 6) * np.sum(Pi)
    return u

In [6]:
max_uscore = utility_score(data['date'].values, data['weight'].values,
                                data['resp'].values, data['action'].values)

## CV - Purged Group Time Series Split

In [7]:
import numpy as np
from sklearn.model_selection import KFold
from sklearn.model_selection._split import _BaseKFold, indexable, _num_samples
from sklearn.utils.validation import _deprecate_positional_args

# modified code for group gaps; source
# https://github.com/getgaurav2/scikit-learn/blob/d4a3af5cc9da3a76f0266932644b884c99724c57/sklearn/model_selection/_split.py#L2243
class PurgedGroupTimeSeriesSplit(_BaseKFold):
    """Time Series cross-validator variant with non-overlapping groups.
    Allows for a gap in groups to avoid potentially leaking info from
    train into test if the model has windowed or lag features.
    Provides train/test indices to split time series data samples
    that are observed at fixed time intervals according to a
    third-party provided group.
    In each split, test indices must be higher than before, and thus shuffling
    in cross validator is inappropriate.
    This cross-validation object is a variation of :class:`KFold`.
    In the kth split, it returns first k folds as train set and the
    (k+1)th fold as test set.
    The same group will not appear in two different folds (the number of
    distinct groups has to be at least equal to the number of folds).
    Note that unlike standard cross-validation methods, successive
    training sets are supersets of those that come before them.
    Read more in the :ref:`User Guide <cross_validation>`.
    Parameters
    ----------
    n_splits : int, default=5
        Number of splits. Must be at least 2.
    max_train_group_size : int, default=Inf
        Maximum group size for a single training set.
    group_gap : int, default=None
        Gap between train and test
    max_test_group_size : int, default=Inf
        We discard this number of groups from the end of each train split
    """

    @_deprecate_positional_args
    def __init__(self,
                 n_splits=5,
                 *,
                 max_train_group_size=np.inf,
                 max_test_group_size=np.inf,
                 group_gap=None,
                 verbose=False
                 ):
        super().__init__(n_splits, shuffle=False, random_state=None)
        self.max_train_group_size = max_train_group_size
        self.group_gap = group_gap
        self.max_test_group_size = max_test_group_size
        self.verbose = verbose

    def split(self, X, y=None, groups=None):
        """Generate indices to split data into training and test set.
        Parameters
        ----------
        X : array-like of shape (n_samples, n_features)
            Training data, where n_samples is the number of samples
            and n_features is the number of features.
        y : array-like of shape (n_samples,)
            Always ignored, exists for compatibility.
        groups : array-like of shape (n_samples,)
            Group labels for the samples used while splitting the dataset into
            train/test set.
        Yields
        ------
        train : ndarray
            The training set indices for that split.
        test : ndarray
            The testing set indices for that split.
        """
        if groups is None:
            raise ValueError(
                "The 'groups' parameter should not be None")
        X, y, groups = indexable(X, y, groups)
        n_samples = _num_samples(X)
        n_splits = self.n_splits
        group_gap = self.group_gap
        max_test_group_size = self.max_test_group_size
        max_train_group_size = self.max_train_group_size
        n_folds = n_splits + 1
        group_dict = {}
        u, ind = np.unique(groups, return_index=True)
        unique_groups = u[np.argsort(ind)]
        n_samples = _num_samples(X)
        n_groups = _num_samples(unique_groups)
        for idx in np.arange(n_samples):
            if (groups[idx] in group_dict):
                group_dict[groups[idx]].append(idx)
            else:
                group_dict[groups[idx]] = [idx]
        if n_folds > n_groups:
            raise ValueError(
                ("Cannot have number of folds={0} greater than"
                 " the number of groups={1}").format(n_folds,
                                                     n_groups))

        group_test_size = min(n_groups // n_folds, max_test_group_size)
        group_test_starts = range(n_groups - n_splits * group_test_size,
                                  n_groups, group_test_size)
        for group_test_start in group_test_starts:
            train_array = []
            test_array = []

            group_st = max(0, group_test_start - group_gap - max_train_group_size)
            for train_group_idx in unique_groups[group_st:(group_test_start - group_gap)]:
                train_array_tmp = group_dict[train_group_idx]
                
                train_array = np.sort(np.unique(
                                      np.concatenate((train_array,
                                                      train_array_tmp)),
                                      axis=None), axis=None)

            train_end = train_array.size
 
            for test_group_idx in unique_groups[group_test_start:
                                                group_test_start +
                                                group_test_size]:
                test_array_tmp = group_dict[test_group_idx]
                test_array = np.sort(np.unique(
                                              np.concatenate((test_array,
                                                              test_array_tmp)),
                                     axis=None), axis=None)

            test_array  = test_array[group_gap:]
            
            
            if self.verbose > 0:
                    pass
                    
            yield [int(i) for i in train_array], [int(i) for i in test_array]

## Building and Training Models

In [8]:
def create_autoencoder(num_input,num_output,noise=0.05):
    i = layers.Input(num_input)
    encoded = layers.BatchNormalization()(i)
    encoded = layers.GaussianNoise(noise)(encoded)
    encoded = layers.Dense(64,activation='relu')(encoded)
    decoded = layers.Dropout(0.2)(encoded)
    decoded = layers.Dense(num_input,name='decoded')(decoded)
    x = layers.Dense(32,activation='relu')(decoded)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.2)(x)
    x = layers.Dense(num_output,activation='sigmoid',name='label_output')(x)
    
    encoder = tf.keras.models.Model(inputs=i,outputs=decoded)
    autoencoder = tf.keras.models.Model(inputs=i,outputs=[decoded,x])

    autoencoder.compile(optimizer=tf.keras.optimizers.Adam(0.001),
                        loss={'decoded':'mse','label_output':'binary_crossentropy'})
    return autoencoder, encoder

### Cross Validation

In [9]:
def create_model_hyperopt(params):
    
    num_layers = params['num_layers']
    num_input = len(features)
    num_output = 1#len(resp_cols)
    hidden_layers = []
    
    dropout_rates = []
    activation = 'swish'
    
    dropout_rates.append(params[f'{num_layers}-dropout0'])
    for i in range (0, num_layers):
        hidden_layers.append(params[f'{num_layers}-layer{i}'])
        dropout_rates.append(params[f'{num_layers}-dropout{i+1}'])

    lrate = params[f'{num_layers}-learning_rate']
    
    inp = layers.Input(shape = (num_input,), name='X_train')
    
    x = inp
    use_encoder = params[f'{num_layers}-use_encoder']
    if use_encoder == True:
        autoencoder, encoder = create_autoencoder(num_input,num_output,noise=0.05)
        encoder.load_weights('../input/janestreet-autoencoder/encoder.h5')
        x = encoder(x)
    use_batchnorm = params[f'{num_layers}-use_batchnorm']
    if use_batchnorm == True:
        x = layers.BatchNormalization()(x)
    x = layers.Dropout(dropout_rates[0])(x)
    
    for i in range(len(hidden_layers)): 
        x = layers.Dense(hidden_layers[i])(x)
        if use_batchnorm == True:
            x = layers.BatchNormalization()(x)
        x = layers.Activation(activation)(x)
        x = layers.Dropout(dropout_rates[i+1])(x)
        
    x = layers.Dense(num_output)(x)
    out = layers.Activation('sigmoid')(x)

    model = tf.keras.models.Model(inputs=inp, outputs=out)
    
    opt = tf.keras.optimizers.Adam(learning_rate = lrate)
    model.compile(optimizer = opt,
                  loss = tf.keras.losses.BinaryCrossentropy(label_smoothing = params[f'{num_layers}-label_smoothing']),
                  metrics = tf.keras.metrics.AUC(name='AUC'))

    return model

In [10]:
def train_model_hyperopt_auc(params):
    
    print ('Parameters: ', params)
    n_folds = 5
    cv = PurgedGroupTimeSeriesSplit(
        n_splits=n_folds,
        max_train_group_size=150,
        group_gap=20,
        max_test_group_size=60
    )
    
    early_stopping = EarlyStopping(monitor='val_AUC',patience=3,min_delta=0.0001, mode='max',
                                    restore_best_weights=True)
    num_epochs = 50
    num_batch = 4096
    scores = []

    for fold, (train_index, test_index) in enumerate(cv.split(data, groups=data['date'].values)):
        print(f'Fold {fold+1} of {n_folds}')
        
        train_set = data.iloc[train_index]
        test_set = data.iloc[test_index]
        
        X_train = train_set.loc[:, features].values
        X_test = test_set.loc[:, features].values
        y_train = np.stack([(train_set[col] > 0).astype('int') for col in resp_cols]).T #Multitarget
        y_test = np.stack([(test_set[col] > 0).astype('int') for col in resp_cols]).T #Multitarget

        model = create_model_hyperopt(params)
        history = model.fit(X_train, y_train,
                            batch_size = num_batch,
                            epochs = num_epochs,
                            verbose = 0,
                            validation_data=(X_test, y_test),
                            callbacks=[early_stopping])

        scores.append(np.max(history.history['val_AUC']))
    
    return {
        'loss': -weighted_average(scores),
        #'loss_variance': np.var(scores, ddof=1),
        'status': STATUS_OK,
        'params': params
    }

In [11]:
def train_model_hyperopt_uscore(params):
    
    print ('Parameters: ', params)
    n_folds = 5
    cv = PurgedGroupTimeSeriesSplit(
        n_splits=n_folds,
        max_train_group_size=150,
        group_gap=20,
        max_test_group_size=60
    )
    
    predictions = np.zeros(len(data['action']))
    
    early_stopping = EarlyStopping(monitor='val_AUC',patience=3,min_delta=0.0001, mode='max',
                                    restore_best_weights=True)
    num_epochs = 50
    num_batch = 4096

    for fold, (train_index, test_index) in enumerate(cv.split(data, groups=data['date'].values)):
        print(f'Fold {fold+1} of {n_folds}')
        
        train_set = data.iloc[train_index]
        test_set = data.iloc[test_index]
        
        X_train = train_set.loc[:, features].values
        X_test = test_set.loc[:, features].values
        y_train = train_set.loc[:, 'action'].values
        y_test = test_set.loc[:, 'action'].values
        #y_train = np.stack([(train_set[col] > 0).astype('int') for col in resp_cols]).T #Multitarget
        #y_test = np.stack([(test_set[col] > 0).astype('int') for col in resp_cols]).T #Multitarget

        model = create_model_hyperopt(params)
        history = model.fit(X_train, y_train,
                            batch_size = num_batch,
                            epochs = num_epochs,
                            verbose = 0,
                            validation_data=(X_test, y_test),
                            callbacks=[early_stopping])

        predictions[test_index] += model.predict(X_test).flatten()
    
    y_pred = ((predictions > 0.501)*1)
    cv_uscore = utility_score(data['date'].values, data['weight'].values, data['resp'].values, y_pred)
    print(f'Utility Score: {cv_uscore}')
    
    return (max_uscore - cv_uscore)

## Search Spaces

In [12]:
# # 3 layers - tuning hidden units only

# space_topology_3 = {
#     'num_layers': 3,
#     '3-learning_rate': 0.001,
#     '3-layer0': scope.int(hp.quniform('3-layer0',64,1024,16)),
#     '3-layer1': scope.int(hp.quniform('3-layer1',64,1024,16)),
#     '3-layer2': scope.int(hp.quniform('3-layer2',64,1024,16)),
#     '3-dropout0': 0.0,#hp.uniform('3-dropout0', 0.0, 0.2),
#     '3-dropout1': 0.0,#hp.uniform('3-dropout1', 0.2, 0.5),
#     '3-dropout2': 0.0,#hp.uniform('3-dropout2', 0.2, 0.5),
#     '3-dropout3': 0.0,#hp.uniform('3-dropout3', 0.2, 0.5),
#     '3-label_smoothing': 0.0,#hp.uniform('3-label_smoothing', 0.0, 0.5),
#     '3-use_batchnorm': False,
#     '3-use_encoder': False,
# }
# trials_topology_3 = Trials()
# best_topology_3 = fmin(train_model_hyperopt_uscore, space_topology_3, algo=tpe.suggest, max_evals=30, trials=trials_topology_3)

In [13]:
## 3 layers - tuning regularization only

# space_topology_3 = {
#     'num_layers': 3,
#     '3-learning_rate': 0.001,
#     '3-layer0': 560,
#     '3-layer1': 592,
#     '3-layer2': 1008,
#     '3-dropout0': hp.uniform('3-dropout0', 0.0, 0.2),
#     '3-dropout1': hp.uniform('3-dropout1', 0.2, 0.5),
#     '3-dropout2': hp.uniform('3-dropout2', 0.2, 0.5),
#     '3-dropout3': hp.uniform('3-dropout3', 0.2, 0.5),
#     '3-label_smoothing': hp.uniform('3-label_smoothing', 0.0, 0.5),
#     '3-use_batchnorm': False,
#     '3-use_encoder': False,
# }
# trials_topology_3 = Trials()
# best_topology_3 = fmin(train_model_hyperopt_uscore, space_topology_3, algo=tpe.suggest, max_evals=30, trials=trials_topology_3)

In [14]:
# print("\n3 Layers:\n")
# print(space_eval(space_topology_3, best_topology_3))
# del space_topology_3, best_topology_3, trials_topology_3

In [15]:
# # 4 layers - tuning hidden units only

# space_topology_4 = {
#     'num_layers': 4,
#     '4-learning_rate': 0.001,
#     '4-layer0': scope.int(hp.quniform('4-layer0',64,1024,16)),
#     '4-layer1': scope.int(hp.quniform('4-layer1',64,1024,16)),
#     '4-layer2': scope.int(hp.quniform('4-layer2',64,1024,16)),
#     '4-layer3': scope.int(hp.quniform('4-layer3',64,1024,16)),
#     '4-dropout0': 0.0,#hp.uniform('4-dropout0', 0.0, 0.2),
#     '4-dropout1': 0.0,#hp.uniform('4-dropout1', 0.2, 0.5),
#     '4-dropout2': 0.0,#hp.uniform('4-dropout2', 0.2, 0.5),
#     '4-dropout3': 0.0,#hp.uniform('4-dropout3', 0.2, 0.5),
#     '4-dropout4': 0.0,#hp.uniform('4-dropout4', 0.2, 0.5),
#     '4-label_smoothing': 0.0,#hp.uniform('4-label_smoothing', 0.0, 0.5),
#     '4-use_batchnorm': False,
#     '4-use_encoder': False,
# }
# trials_topology_4 = Trials()
# best_topology_4 = fmin(train_model_hyperopt_uscore, space_topology_4, algo=tpe.suggest, max_evals=15, trials=trials_topology_4)

In [16]:
## 4 layers - tuning regularization only

# space_topology_4 = {
#     'num_layers': 4,
#     '4-learning_rate': 0.001,
#     '4-layer0': 256,
#     '4-layer1': 288,
#     '4-layer2': 720,
#     '4-layer3': 240,
#     '4-dropout0': hp.uniform('4-dropout0', 0.0, 0.2),
#     '4-dropout1': hp.uniform('4-dropout1', 0.2, 0.5),
#     '4-dropout2': hp.uniform('4-dropout2', 0.2, 0.5),
#     '4-dropout3': hp.uniform('4-dropout3', 0.2, 0.5),
#     '4-dropout4': hp.uniform('4-dropout4', 0.2, 0.5),
#     '4-label_smoothing': hp.uniform('4-label_smoothing', 0.0, 0.5),
#     '4-use_batchnorm': False,
#     '4-use_encoder': False,
# }
# trials_topology_4 = Trials()
# best_topology_4 = fmin(train_model_hyperopt_uscore, space_topology_4, algo=tpe.suggest, max_evals=15, trials=trials_topology_4)

In [17]:
# print("\n4 Layers:\n")
# print(space_eval(space_topology_4, best_topology_4))
# del space_topology_4, best_topology_4, trials_topology_4

In [18]:
# # 5 layers - tuning hidden units only

# space_topology_5 = {
#     'num_layers': 5,
#     '5-learning_rate': 0.001,
#     '5-layer0': scope.int(hp.quniform('5-layer0',64,1024,8)),
#     '5-layer1': scope.int(hp.quniform('5-layer1',64,1024,8)),
#     '5-layer2': scope.int(hp.quniform('5-layer2',64,1024,8)),
#     '5-layer3': scope.int(hp.quniform('5-layer3',64,1024,8)),
#     '5-layer4': scope.int(hp.quniform('5-layer4',64,1024,8)),
#     '5-dropout0': 0.0,#hp.uniform('5-dropout0', 0.0, 0.2),
#     '5-dropout1': 0.0,#hp.uniform('5-dropout1', 0.2, 0.5),
#     '5-dropout2': 0.0,#hp.uniform('5-dropout2', 0.2, 0.5),
#     '5-dropout3': 0.0,#hp.uniform('5-dropout3', 0.2, 0.5),
#     '5-dropout4': 0.0,#hp.uniform('5-dropout4', 0.2, 0.5),
#     '5-dropout5': 0.0,#hp.uniform('5-dropout5', 0.2, 0.5),
#     '5-label_smoothing': 0.0,#hp.uniform('5-label_smoothing', 0.0, 0.5),
#     '5-use_batchnorm': True,
#     '5-use_encoder': True,
# }
# trials_topology_5 = Trials()
# best_topology_5 = fmin(train_model_hyperopt_uscore, space_topology_5, algo=tpe.suggest, max_evals=15, trials=trials_topology_5)

In [19]:
# 5 layers - tuning regularization only
space_topology_5 = {
    'num_layers': 5,
    '5-learning_rate': 0.001,
    '5-layer0': 416,
    '5-layer1': 504,
    '5-layer2': 256,
    '5-layer3': 568,
    '5-layer4': 368,
    '5-dropout0': hp.uniform('5-dropout0', 0.0, 0.2),
    '5-dropout1': hp.uniform('5-dropout1', 0.2, 0.5),
    '5-dropout2': hp.uniform('5-dropout2', 0.2, 0.5),
    '5-dropout3': hp.uniform('5-dropout3', 0.2, 0.5),
    '5-dropout4': hp.uniform('5-dropout4', 0.2, 0.5),
    '5-dropout5': hp.uniform('5-dropout5', 0.2, 0.5),
    '5-label_smoothing': hp.uniform('5-label_smoothing', 0.0, 0.5),
    '5-use_batchnorm': True,
    '5-use_encoder': True,
}
trials_topology_5 = Trials()
best_topology_5 = fmin(train_model_hyperopt_uscore, space_topology_5, algo=tpe.suggest, max_evals=15, trials=trials_topology_5)

Parameters: 
{'5-dropout0': 0.1747245172397526, '5-dropout1': 0.46609158038970466, '5-dropout2': 0.42073578763921193, '5-dropout3': 0.23349072903009116, '5-dropout4': 0.413193305598212, '5-dropout5': 0.4562599801134036, '5-label_smoothing': 0.39645093182491814, '5-layer0': 416, '5-layer1': 504, '5-layer2': 256, '5-layer3': 568, '5-layer4': 368, '5-learning_rate': 0.001, '5-use_batchnorm': True, '5-use_encoder': True, 'num_layers': 5}
Fold 1 of 5
  0%|          | 0/15 [00:04<?, ?trial/s, best loss=?]

2022-07-29 16:59:47.050876: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-29 16:59:47.157646: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-29 16:59:47.158429: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-07-29 16:59:47.159566: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Fold 2 of 5
Fold 3 of 5
Fold 4 of 5
Fold 5 of 5
Utility Score: 1510.675177442345
Parameters: 
{'5-dropout0': 0.16542228223546193, '5-dropout1': 0.47034933045585237, '5-dropout2': 0.3235030229079465, '5-dropout3': 0.482234995732669, '5-dropout4': 0.39713826728060597, '5-dropout5': 0.41162054551107996, '5-label_smoothing': 0.06758285046886642, '5-layer0': 416, '5-layer1': 504, '5-layer2': 256, '5-layer3': 568, '5-layer4': 368, '5-learning_rate': 0.001, '5-use_batchnorm': True, '5-use_encoder': True, 'num_layers': 5}
Fold 1 of 5
Fold 2 of 5
Fold 3 of 5
Fold 4 of 5
Fold 5 of 5
Utility Score: 1215.0419302761659
Parameters: 
{'5-dropout0': 0.0022263254635593423, '5-dropout1': 0.2417803329100388, '5-dropout2': 0.3949217589967401, '5-dropout3': 0.45549560651758625, '5-dropout4': 0.45087772369705753, '5-dropout5': 0.3477748117739772, '5-label_smoothing': 0.3072349409545522, '5-layer0': 416, '5-layer1': 504, '5-layer2': 256, '5-layer3': 568, '5-layer4': 368, '5-learning_rate': 0.001, '5-use_batc

In [20]:
print("\n5 Layers:\n")
print(space_eval(space_topology_5, best_topology_5))
del space_topology_5, best_topology_5, trials_topology_5


5 Layers:

{'5-dropout0': 0.1747245172397526, '5-dropout1': 0.46609158038970466, '5-dropout2': 0.42073578763921193, '5-dropout3': 0.23349072903009116, '5-dropout4': 0.413193305598212, '5-dropout5': 0.4562599801134036, '5-label_smoothing': 0.39645093182491814, '5-layer0': 416, '5-layer1': 504, '5-layer2': 256, '5-layer3': 568, '5-layer4': 368, '5-learning_rate': 0.001, '5-use_batchnorm': True, '5-use_encoder': True, 'num_layers': 5}


In [21]:
# 5 layers - tuning regularization only
space_topology_5 = {
    'num_layers': 5,
    '5-learning_rate': 0.001,
    '5-layer0': 224,
    '5-layer1': 440,
    '5-layer2': 688,
    '5-layer3': 400,
    '5-layer4': 112,
    '5-dropout0': hp.uniform('5-dropout0', 0.0, 0.2),
    '5-dropout1': hp.uniform('5-dropout1', 0.2, 0.5),
    '5-dropout2': hp.uniform('5-dropout2', 0.2, 0.5),
    '5-dropout3': hp.uniform('5-dropout3', 0.2, 0.5),
    '5-dropout4': hp.uniform('5-dropout4', 0.2, 0.5),
    '5-dropout5': hp.uniform('5-dropout5', 0.2, 0.5),
    '5-label_smoothing': hp.uniform('5-label_smoothing', 0.0, 0.5),
    '5-use_batchnorm': False,
    '5-use_encoder': False,
}
trials_topology_5 = Trials()
best_topology_5 = fmin(train_model_hyperopt_uscore, space_topology_5, algo=tpe.suggest, max_evals=15, trials=trials_topology_5)

Parameters: 
{'5-dropout0': 0.016200509288124065, '5-dropout1': 0.21823260665158842, '5-dropout2': 0.23217007093887884, '5-dropout3': 0.3060109961056432, '5-dropout4': 0.3735499891495283, '5-dropout5': 0.4221240339272655, '5-label_smoothing': 0.4314995350872904, '5-layer0': 224, '5-layer1': 440, '5-layer2': 688, '5-layer3': 400, '5-layer4': 112, '5-learning_rate': 0.001, '5-use_batchnorm': False, '5-use_encoder': False, 'num_layers': 5}
Fold 1 of 5
Fold 2 of 5
Fold 3 of 5
Fold 4 of 5
Fold 5 of 5
Utility Score: 1410.5913270733429
Parameters: 
{'5-dropout0': 0.09744547850890897, '5-dropout1': 0.3054659796747164, '5-dropout2': 0.4632370655157195, '5-dropout3': 0.3421458214020789, '5-dropout4': 0.3632368821344707, '5-dropout5': 0.324276493237409, '5-label_smoothing': 0.17142181044191201, '5-layer0': 224, '5-layer1': 440, '5-layer2': 688, '5-layer3': 400, '5-layer4': 112, '5-learning_rate': 0.001, '5-use_batchnorm': False, '5-use_encoder': False, 'num_layers': 5}
Fold 1 of 5
Fold 2 of 5
Fol

In [22]:
print("\n5 Layers:\n")
print(space_eval(space_topology_5, best_topology_5))
del space_topology_5, best_topology_5, trials_topology_5


5 Layers:

{'5-dropout0': 0.0384658221414784, '5-dropout1': 0.3713630172398603, '5-dropout2': 0.4522458949287438, '5-dropout3': 0.4404002213547826, '5-dropout4': 0.20424755544729625, '5-dropout5': 0.43797286980905237, '5-label_smoothing': 0.1881616587283278, '5-layer0': 224, '5-layer1': 440, '5-layer2': 688, '5-layer3': 400, '5-layer4': 112, '5-learning_rate': 0.001, '5-use_batchnorm': False, '5-use_encoder': False, 'num_layers': 5}
